In [14]:
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
from create_data import create_sparse_data
import torch
from scipy.optimize import minimize
import os

def calc_probabilities(file_name, mean_seed_mode=False, seed_vec=[]):
    # Load the data
    df = pd.read_csv(file_name)
    # print(df)
    # data_seed_values = [0, 300, 600, 900]
    if len(seed_vec)==0:
        data_seed_values = df['data_seed'].unique()
    else:
        data_seed_values = seed_vec
    depth_vec = df['depth'].unique()
    loss_thr_vec = [(0, 1)]
    res_df = pd.DataFrame()
    for loss_thr_ind, loss_thr in enumerate(loss_thr_vec):
        for depth in depth_vec:
            for dist in ["Normal", "Normal_uv", "Laplace"]:
                if depth>2 and dist!="Normal_uv":
                    continue
                p2_vec = []
                # if mean_seed_mode:
                #     df_dist = df[(df['dist']==dist) & (df['depth']==depth) & (df['loss_min_thr']==loss_thr[0]) & (df['loss_max_thr']==loss_thr[1]) & (df['status']=='Success')]
                #     if len(df_dist)>0:
                #             p1 = len(df_dist[df_dist['test_acc']>0])/((df_dist['iter']+1).sum())
                #             p2 = (df_dist[df_dist['test_acc']>=0]['test_acc']).mean()
                #             cur_res_df = pd.DataFrame({'dist': dist, 'depth': depth, 'p1': p1, 'p2': p2}, index=[0])
                #             res_df = pd.concat([res_df, cur_res_df], axis=0)
                # else:
                seed_counter = 0
                for seed_ind, seed in enumerate(data_seed_values):
                    df_dist = df[(df['dist']==dist) & (df['data_seed']==seed) & (df['depth']==depth) & (df['loss_min_thr']==loss_thr[0]) & (df['loss_max_thr']==loss_thr[1]) & (df['status']=='Success')]                        
                    if len(df_dist)>0:
                        p1 = len(df_dist[df_dist['test_acc']>0])/((df_dist['iter']+1).sum())
                        p2 = (df_dist[df_dist['test_acc']>=0]['test_acc']).mean()
                        p2_vec.append(p2)
                        seed_counter+=1
                    if not mean_seed_mode:
                        cur_res_df = pd.DataFrame({'seed': seed, 'dist': dist, 'depth': depth, 'p1': p1, 'p2': p2}, index=[0])
                        res_df = pd.concat([res_df, cur_res_df], axis=0)
                if mean_seed_mode and seed_counter>0:
                    p2_mean = np.mean(p2_vec)
                    p2_std = np.std(p2_vec)
                    cur_res_df = pd.DataFrame({'dist': dist, 'depth': depth, 'p1': p1, 'p2_mean': p2_mean, 'p2_std': p2_std, 'seed_counter': seed_counter}, index=[0])
                    res_df = pd.concat([res_df, cur_res_df], axis=0)
    return res_df


In [15]:


def constraint(v, x, y):
    return np.multiply(y, np.matmul(x, v.reshape(-1, 1)).squeeze())-1


def solver(x, y, w_0, obj='Q', optim_tol=1e-5):
    x0 = (w_0).reshape(-1, )

    cons = {'type': 'ineq', 'fun': lambda v: constraint(v, x, y)}

    if obj == 'L1':
        objective = lambda v: np.linalg.norm(v.squeeze(), ord=1)
    elif obj == 'L2':
        objective = lambda v: np.linalg.norm(v.squeeze(), ord=2)
    else:
        raise ValueError('objective not supported.')

    sol = minimize(
        fun=objective,
        x0=x0,
        constraints=cons,
        tol=optim_tol,
        method='SLSQP',
        options={
            'maxiter': 1000000,
            'disp': False
        }
    )
    is_failed = (not sol.success)
    if is_failed:
        raise RuntimeError('Minimization Failed.')

    return sol.x

def calc_test_acc(x_test, y_test, w):
    y_hat = torch.matmul(x_test, torch.tensor(w).float())
    predicton = (y_hat*y_test>0).float()
    test_acc = torch.sum(predicton)/len(y_test)
    return test_acc

In [16]:
def calc_baselines(file_name):
    # Load the results
    df = pd.read_csv(file_name)
    N_train = df['N_train'].unique()[0]
    N_test = df['N_test'].unique()[0]
    d = df['d'].unique()[0]
    r = df['r'].unique()[0]
    mu = df['mu'].unique()[0]
    dataset = df['dataset'].unique()[0]
    data_seed_values = df['data_seed'].unique()
    test_acc_across_seeds_l0 = []
    test_acc_across_seeds_l1 = []
    for seed_ind, seed in enumerate(data_seed_values):
        x_train, y_train, x_test, y_test = create_sparse_data(n_train=N_train, n_test=N_test, r=r, dataset=dataset, normalized_flag=False, seed=seed, mu=mu, d=d)
        x_train, y_train, x_test, y_test = x_train.cpu(), y_train.cpu(), x_test.cpu(), y_test.cpu() 
        if r==1:
            # l0 baseline
            test_acc_vec_l0 = []
            for ind in range(d):
                w_L0 = np.zeros((d))
                w_L0[ind] = 1
                train_acc = calc_test_acc(x_train, y_train, w_L0)
                if train_acc==1.0:
                    test_acc_vec_l0.append(calc_test_acc(x_test, y_test, w_L0))
            test_acc_l0 = test_acc_vec_l0[np.random.randint(len(test_acc_vec_l0))]
            # save result in csv
            df_res = pd.DataFrame({'N_train': N_train, 'N_test': N_test, 'd': d, 'r': r, 'mu': mu, 'dataset': dataset, 'data_seed': seed, 'type': 'L0', 'test_acc': test_acc_l0.detach().cpu().numpy()}, index=[0])
            baselines_file_name = f"baselines.csv"
            if not os.path.exists(baselines_file_name):
                df_res.to_csv(baselines_file_name, mode='w', index=False)
            # Check if file is empty
            elif os.stat(baselines_file_name).st_size == 0:
                df_res.to_csv(baselines_file_name, mode='a', index=False)
            else:
                df_res.to_csv(baselines_file_name, mode='a', header=False, index=False)
        
        # l1 baseline
        w_0 = np.random.randn(d, 1)
        # Create l1 baseline
        w_L1 = solver(
        x=x_train,
        y=y_train,
        w_0=w_0,
        obj='L1',
        optim_tol=1e-8
        )
        test_acc_across_seeds_l1.append(calc_test_acc(x_test, y_test, w_L1))
        # save result in csv
        df_res = pd.DataFrame({'N_train': N_train, 'N_test': N_test, 'd': d, 'r': r, 'mu': mu, 'dataset': dataset, 'data_seed': seed, 'type': 'L1', 'test_acc': calc_test_acc(x_test, y_test, w_L1).detach().cpu().numpy()}, index=[0])
        baselines_file_name = f"baselines.csv"
        if not os.path.exists(baselines_file_name):
            df_res.to_csv(baselines_file_name, mode='w', index=False)
        # Check if file is empty
        elif os.stat(baselines_file_name).st_size == 0:
            df_res.to_csv(baselines_file_name, mode='a', index=False)
        else:
            df_res.to_csv(baselines_file_name, mode='a', header=False, index=False)

        if seed_ind%10==0:
            print(f"seed={seed}, test_acc={test_acc_across_seeds_l1[-1]}")
    print(test_acc_across_seeds_l1)
    return np.mean(test_acc_across_seeds_l0), np.mean(test_acc_across_seeds_l1) #, np.std(test_acc_across_seeds)


def load_baselines(res_file_name, baselines_file_name):
    # Load the results
    df_res = pd.read_csv(res_file_name)
    df_baselines = pd.read_csv(baselines_file_name)


    N_train = df_res['N_train'].unique()[0]
    N_test = df_res['N_test'].unique()[0]
    d = df_res['d'].unique()[0]
    r = df_res['r'].unique()[0]
    mu = df_res['mu'].unique()[0]
    dataset = df_res['dataset'].unique()[0]
    l1_baseline = df_baselines.loc[(df_baselines['type']=='L1') & (df_baselines['N_train']==N_train) & (df_baselines['N_test']==N_test) & (df_baselines['d']==d) & (df_baselines['r']==r) & (df_baselines['mu']==mu) & (df_baselines['dataset']==dataset)]['test_acc'].mean()
    
    if r==1:
        l0_baseline = df_baselines.loc[(df_baselines['type']=='L0') & (df_baselines['N_train']==N_train) & (df_baselines['N_test']==N_test) & (df_baselines['d']==d) & (df_baselines['r']==r) & (df_baselines['mu']==mu) & (df_baselines['dataset']==dataset)]['test_acc'].mean()
    else:
        l0_baseline = -1
    return l1_baseline, l0_baseline

In [21]:
N_train_vec = [16]
d_vec = [128]
r = 1

res_total_df = pd.DataFrame()
for N_train in N_train_vec:
    for d in d_vec:
        file_name = f"results_new_1005_prob_N_train_{N_train}_d_{d}_r_{r}.csv"
        baselines_file_name = f"baselines.csv"
        baselinel1, baselinel0 = calc_baselines(file_name)
        baselinel1, baselinel0 = load_baselines(file_name, baselines_file_name)
        try:
            print(file_name)
            res_df = calc_probabilities(file_name, mean_seed_mode=True)
            res_df = pd.concat([pd.DataFrame({'N_train': N_train, 'd': d, 'r': r}, index=[0]), res_df], axis=1)
            res_total_df = pd.concat([res_total_df, res_df], axis=0)
            res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L0', 'p2_mean': baselinel0}, index=[0])], axis=0)
            res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L1', 'p2_mean': baselinel1}, index=[0])], axis=0)
        except:
            pass

seed=0, test_acc=0.800000011920929
[tensor(0.8000), tensor(1.), tensor(0.9000), tensor(0.6000), tensor(1.), tensor(0.8000), tensor(0.8000), tensor(0.9000), tensor(0.6000), tensor(0.7000)]
results_new_1005_prob_N_train_16_d_128_r_1.csv


/home/mor/anaconda3/envs/myenv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mor/anaconda3/envs/myenv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [22]:
res_total_df

""


In [ ]:
N_train_vec = [12]
d_vec = [128]
r = 1

res_total_df = pd.DataFrame()
for N_train in N_train_vec:
    for d in d_vec:
        file_name = f"results_new_prob_N_train_{N_train}_d_{d}.csv"
        baselines_file_name = f"baselines.csv"
        # baseline = calc_baselines(file_name)
        baselinel1, baselinel0 = load_baselines(file_name, baselines_file_name)
        try:
            print(file_name)
            res_df = calc_probabilities(file_name, mean_seed_mode=True)
            res_df = pd.concat([pd.DataFrame({'N_train': N_train, 'd': d, 'r': r}, index=[0]), res_df], axis=1)
            res_total_df = pd.concat([res_total_df, res_df], axis=0)
            res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L0', 'p2_mean': baselinel0}, index=[0])], axis=0)
            res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L1', 'p2_mean': baselinel1}, index=[0])], axis=0)
        except:
            pass
			
N_train_vec = [12]
d_vec = [128]
r = 5

res_total_df = pd.DataFrame()
for N_train in N_train_vec:
    for d in d_vec:
        file_name = f"results_new_prob_N_train_{N_train}_d_{d}_r_{r}.csv"
        # baseline = calc_baselines(file_name)
        baselines_file_name = f"baselines.csv"
        baselinel1, _ = load_baselines(file_name, baselines_file_name)
        try:
            print(file_name)
            res_df = calc_probabilities(file_name, mean_seed_mode=True)
            res_df = pd.concat([pd.DataFrame({'N_train': N_train, 'd': d, 'r': r}, index=[0]), res_df], axis=1)
            res_total_df = pd.concat([res_total_df, res_df], axis=0)
            res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L1', 'p2_mean': baselinel1}, index=[0])], axis=0)
        except:
            pass
			
N_train_vec = [12]
d_vec = [128]
r = 2

res_total_df = pd.DataFrame()
for N_train in N_train_vec:
    for d in d_vec:
        file_name = f"results_new_prob_N_train_{N_train}_d_{d}_r_{r}.csv"
        # baseline = calc_baselines(file_name)
        baselines_file_name = f"baselines.csv"
        baselinel1, _ = load_baselines(file_name, baselines_file_name)
        try:
            print(file_name)
            res_df = calc_probabilities(file_name, mean_seed_mode=True)
            res_df = pd.concat([pd.DataFrame({'N_train': N_train, 'd': d, 'r': r}, index=[0]), res_df], axis=1)
            res_total_df = pd.concat([res_total_df, res_df], axis=0)
            res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L1', 'p2_mean': baselinel1}, index=[0])], axis=0)
        except:
            pass
			
## Generalization Vs Depth

N_train_vec = [12]
d_vec = [128]
r = 1

res_total_df = pd.DataFrame()
for N_train in N_train_vec:
    for d in d_vec:
        file_name = f"results_new_prob_N_train_{N_train}_d_{d}_r_{r}.csv"
        # baseline = calc_baselines(file_name)
        baselines_file_name = f"baselines.csv"
        baselinel1, _ = load_baselines(file_name, baselines_file_name)
        # try:
        print(file_name)
        res_df = calc_probabilities(file_name, mean_seed_mode=True)
        res_df = pd.concat([pd.DataFrame({'N_train': N_train, 'd': d, 'r': r}, index=[0]), res_df], axis=1)
        res_total_df = pd.concat([res_total_df, res_df], axis=0)
        res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L1', 'p2_mean': baselinel1}, index=[0])], axis=0)
        # except:
            # pass
			
## r=2

N_train_vec = [12]
d_vec = [128]
r = 2

res_total_df = pd.DataFrame()
for N_train in N_train_vec:
    for d in d_vec:
        file_name = f"results_new_prob_N_train_{N_train}_d_{d}_r_{r}.csv"
        # baseline = calc_baselines(file_name)
        baselines_file_name = f"baselines.csv"
        baselinel1, _ = load_baselines(file_name, baselines_file_name)
        # try:
        print(file_name)
        res_df = calc_probabilities(file_name, mean_seed_mode=True, seed_vec=np.arange(0,10).tolist())
        res_df = pd.concat([pd.DataFrame({'N_train': N_train, 'd': d, 'r': r}, index=[0]), res_df], axis=1)
        res_total_df = pd.concat([res_total_df, res_df], axis=0)
        res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L1', 'p2_mean': baselinel1}, index=[0])], axis=0)
        # except:
            # pass
			
res_total_df

plt.plot(res_total_df[res_total_df['dist']=='Normal_uv']['depth'], res_total_df[res_total_df['dist']=='Normal_uv']['p2_mean'], 'o')
plt.xlabel("Depth")
plt.ylabel("Mean Test Accuracy")

N_train_vec = [32]
d_vec = [128]
r = 2

res_total_df = pd.DataFrame()
for N_train in N_train_vec:
    for d in d_vec:
        file_name = f"results_new_prob_N_train_{N_train}_d_{d}_r_{r}.csv"
        # baseline = calc_baselines(file_name)
        baselines_file_name = f"baselines.csv"
        baselinel1, _ = load_baselines(file_name, baselines_file_name)
        # try:
        print(file_name)
        res_df = calc_probabilities(file_name, mean_seed_mode=True, seed_vec=np.arange(0,10).tolist())
        res_df = pd.concat([pd.DataFrame({'N_train': N_train, 'd': d, 'r': r}, index=[0]), res_df], axis=1)
        res_total_df = pd.concat([res_total_df, res_df], axis=0)
        res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L1', 'p2_mean': baselinel1}, index=[0])], axis=0)
        # except:
            # pass
			
res_total_df

In [28]:
N_train_vec = [16]
d_vec = [128]
r = 1

res_total_df = pd.DataFrame()
for N_train in N_train_vec:
    for d in d_vec:
        file_name = f"results_new_1005_prob_N_train_{N_train}_d_{d}_r_{r}.csv"
        # baseline = calc_baselines(file_name)
        baselines_file_name = f"baselines.csv"
        baselinel1, _ = load_baselines(file_name, baselines_file_name)
        # try:
        print(file_name)
        res_df = calc_probabilities(file_name, mean_seed_mode=True, seed_vec=np.arange(0,10).tolist())
        # res_df = pd.concat([pd.DataFrame({'N_train': N_train, 'd': d, 'r': r}, index=[0]), res_df], axis=1)
        # res_total_df = pd.concat([res_total_df, res_df], axis=0)
        # res_total_df = pd.concat([res_total_df, pd.DataFrame({'N_train': N_train, 'd': d, 'r': r, 'dist': 'L1', 'p2_mean': baselinel1}, index=[0])], axis=0)
        # except:
            # pass
			
res_df

results_new_1005_prob_N_train_16_d_128_r_1.csv


,dist,depth,p1,p2_mean,p2_std,seed_counter
0,Normal_uv,2,0.000031,0.497536,0.030176,10
0,Normal_uv,4,0.000049,0.675466,0.062799,10
0,Normal_uv,6,0.000081,0.827787,0.056976,10
0,Normal_uv,8,0.000180,0.872884,0.049011,10
0,Normal_uv,10,0.000270,0.898529,0.043779,10
0,Normal_uv,15,0.000514,0.932552,0.033749,10
0,Normal_uv,20,0.000765,0.947810,0.028607,10
0,Normal_uv,25,0.000883,0.954295,0.025231,10
0,Normal_uv,30,0.001044,0.961099,0.023135,10
0,Normal_uv,35,0.001235,0.965036,0.022091,10
